# COVID 19 Basic Visualization

In [1]:
import os
import pandas as pd
from ipywidgets import interact, Layout
import ipywidgets as widgets
import matplotlib.dates as mdates
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import datetime as dt
import calmap

In [2]:
# df_confirmed_o = pd.read_csv(r'data/time_series_covid19_confirmed_global.csv')
df_confirmed_o = pd.read_csv('https://coviddata.github.io/coviddata/v1/countries/cases.csv')
df_confirmed = df_confirmed_o.groupby(['Country'], as_index=False).sum()
# df_confirmed.head()

In [3]:
df_deaths_o = pd.read_csv('https://coviddata.github.io/coviddata/v1/countries/deaths.csv')
df_deaths = df_deaths_o.groupby(['Country'], as_index=False).sum()
# df_deaths.head()

In [4]:
df_recovered_o = pd.read_csv('https://coviddata.github.io/coviddata/v1/countries/recoveries.csv')
df_recovered = df_recovered_o.groupby(['Country'], as_index=False).sum()
# df_recovered.head()

In [5]:
def f2(x):
    dates = [dt.datetime.strptime(d,'%Y-%m-%d').date() for d in list(df_confirmed.columns[1:])]
    st_val = dates.index(date_sel2.value[0]) + 1
    end_val = dates.index(date_sel2.value[1]) + 2
    
    x = [dt.datetime.strptime(d,'%Y-%m-%d').date() for d in list(df_confirmed.columns[st_val:end_val])]    
    y1 = df_confirmed[df_confirmed['Country'] == country_sel2.value].iloc[0][st_val:end_val]
    y1.index = pd.to_datetime(y1.index)
    y2 = df_deaths[df_deaths['Country'] == country_sel2.value].iloc[0][st_val:end_val]
    y2.index = pd.to_datetime(y1.index)
    y3 = df_recovered[df_recovered['Country'] == country_sel2.value].iloc[0][st_val:end_val]
    y3.index = pd.to_datetime(y1.index)
    
    fig5.data[0].x = y1.index
    fig5.data[0].y = y1

    fig5.data[1].x = y2.index
    fig5.data[1].y = y2

    fig5.data[2].x = y3.index
    fig5.data[2].y = y3
    
    if uniform_sel.value:
        h = max([y1.max(), y2.max(), y3.max()])
        fig5.update_yaxes(range=[0, h])
    else:
        fig5.update_yaxes(range=[0, y1.max()], row=1, col=1)
        fig5.update_yaxes(range=[0, y2.max()], row=1, col=2)
        fig5.update_yaxes(range=[0, y3.max()], row=1, col=3)


    
    
country_sel2 = widgets.Dropdown(
    options=list(df_confirmed['Country'].unique()),
    description='Country:',
    value='India',
    disabled=False,
)

dates = [dt.datetime.strptime(d,'%Y-%m-%d').date() for d in list(df_confirmed.columns[1:])]
dates_i = range(len(dates))
options = [(i.strftime(' %d %b %y '), i) for i in dates]
date_sel2 = widgets.SelectionRangeSlider(
    options=options,
    index=(0, len(dates)-1),
    description='Months',
    disabled=False,
    continuous_update=False,
    layout=Layout(width='70%', height='80px')
)

uniform_sel = widgets.Checkbox(
    value=True,
    description='Scale uniformly',
    disabled=False
)
    
country_sel2.observe(f2, names='value')
date_sel2.observe(f2, names='value')
uniform_sel.observe(f2, names='value')


x = [dt.datetime.strptime(d,'%Y-%m-%d').date() for d in list(df_confirmed.columns[1:])]
    
y1 = df_confirmed[df_confirmed['Country'] == 'India'].iloc[0][1:]
y1.index = pd.to_datetime(y1.index)
y2 = df_deaths[df_deaths['Country'] == 'India'].iloc[0][1:]
y2.index = pd.to_datetime(y2.index)
y3 = df_recovered[df_recovered['Country'] == 'India'].iloc[0][1:]
y3.index = pd.to_datetime(y3.index)

fig5 = make_subplots(rows=1, cols=3, subplot_titles=("Confirmed", "Deaths", "Recovered"))

fig5.add_trace(
    go.Scatter(
        x=y1.index, y=y1,
        line=dict(color='blue', width=2),
        name="Confirmed"
    ),
    row=1, col=1
)

fig5.add_trace(
    go.Scatter(
        x=y2.index, y=y2,
        line=dict(color='red', width=2),
        name="Deaths"
    ),
    row=1, col=2
)

fig5.add_trace(
    go.Scatter(
        x=y3.index, y=y3,
        line=dict(color='green', width=2),
        name="Recovered"
    ),
    row=1, col=3
)

fig5 = go.FigureWidget(fig5)
fig5.update_layout(title_text="Cumulative Data on seperate graphs", showlegend=False)

for i in range(3):
    fig5.update_xaxes(title_text="Time", row=1, col=i+1)

fig5.update_yaxes(title_text="Cases", row=1, col=1)
# fig5.update_yaxes(rangemode="nonnegative")
h = max([y1.max(), y2.max(), y3.max()])
fig5.update_yaxes(range=[0, h])

container4 = widgets.HBox([country_sel2, uniform_sel])
cum_sep = widgets.VBox([container4,
              date_sel2,
              fig5])
cum_sep

In [6]:
def f(x):
    dates = [dt.datetime.strptime(d,'%Y-%m-%d').date() for d in list(df_confirmed.columns[1:])]
    st_val = dates.index(date_sel.value[0]) + 1
    end_val = dates.index(date_sel.value[1]) + 2
    
    x = [dt.datetime.strptime(d,'%Y-%m-%d').date() for d in list(df_confirmed.columns[st_val:end_val])]    
    y1 = df_confirmed[df_confirmed['Country'] == country_sel.value].iloc[0][st_val:end_val]
    y1.index = pd.to_datetime(y1.index)
    y2 = df_deaths[df_deaths['Country'] == country_sel.value].iloc[0][st_val:end_val]
    y2.index = pd.to_datetime(y1.index)
    y3 = df_recovered[df_recovered['Country'] == country_sel.value].iloc[0][st_val:end_val]
    y3.index = pd.to_datetime(y1.index)

    if confirmed_sel.value:
        fig4.data[0].x = y1.index
        fig4.data[0].y = y1
    else:
        fig4.data[0].x = None
        fig4.data[0].y = None
        
    if death_sel.value:
        fig4.data[1].x = y2.index
        fig4.data[1].y = y2
    else:
        fig4.data[1].x = None
        fig4.data[1].y = None
        
    if recover_sel.value:
        fig4.data[2].x = y3.index
        fig4.data[2].y = y3
    else:
        fig4.data[2].x = None
        fig4.data[2].y = None

    
    
country_sel = widgets.Dropdown(
    options=list(df_confirmed['Country'].unique()),
    description='Country:',
    value='India',
    disabled=False,
)

confirmed_sel = widgets.Checkbox(
    value=True,
    description='Confirmed',
    disabled=False
)

death_sel = widgets.Checkbox(
    value=True,
    description='Death',
    disabled=False
)

recover_sel = widgets.Checkbox(
    value=True,
    description='Recovered',
    disabled=False
)

dates = [dt.datetime.strptime(d,'%Y-%m-%d').date() for d in list(df_confirmed.columns[1:])]
dates_i = range(len(dates))
options = [(i.strftime(' %d %b %y '), i) for i in dates]
date_sel = widgets.SelectionRangeSlider(
    options=options,
    index=(0, len(dates)-1),
    description='Months',
    disabled=False,
    continuous_update=False,
    layout=Layout(width='70%', height='80px')
)
    
country_sel.observe(f, names='value')
confirmed_sel.observe(f, names='value')
death_sel.observe(f, names='value')
recover_sel.observe(f, names='value')
date_sel.observe(f, names='value')


x = [dt.datetime.strptime(d,'%Y-%m-%d').date() for d in list(df_confirmed.columns[1:])]
    
y1 = df_confirmed[df_confirmed['Country'] == 'India'].iloc[0][1:]
y1.index = pd.to_datetime(y1.index)
y2 = df_deaths[df_deaths['Country'] == 'India'].iloc[0][1:]
y2.index = pd.to_datetime(y2.index)
y3 = df_recovered[df_recovered['Country'] == 'India'].iloc[0][1:]
y3.index = pd.to_datetime(y3.index)

fig4 = go.FigureWidget(data=[
    go.Scatter(
        x=y1.index, y=y1,
        line=dict(color='blue', width=2),
        name="Confirmed"
    ),
    go.Scatter(
        x=y2.index, y=y2,
        line=dict(color='red', width=2),
        name="Deaths"
    ),
    go.Scatter(
        x=y3.index, y=y3,
        line=dict(color='green', width=2),
        name="Recovered"
    )
])
fig4.update_layout(
    title='Cumulative data on single graph',
    yaxis_title="Cases",
    xaxis_title="Time",
    legend=dict(x=.8, y=-.5),
#     margin={"r":0,"t":30,"l":20,"b":20}
)

fig4.update_yaxes(rangemode="nonnegative")



container2 = widgets.HBox([country_sel, confirmed_sel, death_sel, recover_sel])
cum_sin = widgets.VBox([container2,
              date_sel,
              fig4])
cum_sin

In [7]:
df_confirmed_n = df_confirmed.copy()

for i in range(len(df_confirmed_n)):
    for j in range(4, len(df_confirmed_n.loc[0])):
        df_confirmed_n.iat[i, j] = df_confirmed_n.iat[i, j] - df_confirmed.iat[i, j-1]
    
# df_confirmed_n.head()

In [13]:
df_deaths_n = df_deaths.copy()

for i in range(len(df_confirmed_n)):
    for j in range(4, len(df_confirmed_n.loc[0])):
        df_deaths_n.iat[i, j] = df_deaths_n.iat[i, j] - df_deaths.iat[i, j-1]
    
# df_deaths_n.head()

,Country,2020-01-22,2020-01-23,2020-01-24,2020-01-25,2020-01-26,2020-01-27,2020-01-28,2020-01-29,2020-01-30,...,2020-05-05,2020-05-06,2020-05-07,2020-05-08,2020-05-09,2020-05-10,2020-05-11,2020-05-12,2020-05-13,2020-05-14
0,Afghanistan,0,0,0,0,0,0,0,0,0,...,5,9,2,3,6,5,2,5,5,4
1,Albania,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Algeria,0,0,0,0,0,0,0,0,0,...,5,6,7,5,6,8,5,8,7,7
3,Andorra,0,0,0,0,0,0,0,0,0,...,1,0,1,0,1,0,0,0,1,0
4,Angola,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [14]:
df_recovered_n = df_recovered.copy()

for i in range(len(df_confirmed_n)):
    for j in range(4, len(df_confirmed_n.loc[0])):
        df_recovered_n.iat[i, j] = df_recovered_n.iat[i, j] - df_recovered.iat[i, j-1]
    
# df_recovered_n.head()

In [15]:
def f3(x):
    dates = [dt.datetime.strptime(d,'%Y-%m-%d').date() for d in list(df_confirmed_n.columns[1:])]
    st_val = dates.index(date_sel3.value[0]) + 1
    end_val = dates.index(date_sel3.value[1]) + 2
    
    x = [dt.datetime.strptime(d,'%Y-%m-%d').date() for d in list(df_confirmed_n.columns[st_val:end_val])]    
    y1 = df_confirmed_n[df_confirmed_n['Country'] == country_sel3.value].iloc[0][st_val:end_val]
    y1.index = pd.to_datetime(y1.index)
    y2 = df_deaths_n[df_deaths_n['Country'] == country_sel3.value].iloc[0][st_val:end_val]
    y2.index = pd.to_datetime(y1.index)
    y3 = df_recovered_n[df_recovered_n['Country'] == country_sel3.value].iloc[0][st_val:end_val]
    y3.index = pd.to_datetime(y1.index)
    
    fig6.data[0].x = y1.index
    fig6.data[0].y = y1

    fig6.data[1].x = y2.index
    fig6.data[1].y = y2

    fig6.data[2].x = y3.index
    fig6.data[2].y = y3

    
    
country_sel3 = widgets.Dropdown(
    options=list(df_confirmed['Country'].unique()),
    description='Country:',
    value='India',
    disabled=False,
)

dates = [dt.datetime.strptime(d,'%Y-%m-%d').date() for d in list(df_confirmed.columns[3:])]
dates_i = range(len(dates))
options = [(i.strftime(' %d %b %y '), i) for i in dates]
date_sel3 = widgets.SelectionRangeSlider(
    options=options,
    index=(0, len(dates)-1),
    description='Months',
    disabled=False,
    continuous_update=False,
    layout=Layout(width='70%', height='80px')
)
    
country_sel3.observe(f3, names='value')
date_sel3.observe(f3, names='value')


x = [dt.datetime.strptime(d,'%Y-%m-%d').date() for d in list(df_confirmed.columns[3:])]
    
y1 = df_confirmed_n[df_confirmed['Country'] == 'India'].iloc[0][3:]
y1.index = pd.to_datetime(y1.index)
y2 = df_deaths_n[df_deaths['Country'] == 'India'].iloc[0][3:]
y2.index = pd.to_datetime(y2.index)
y3 = df_recovered_n[df_recovered['Country'] == 'India'].iloc[0][3:]
y3.index = pd.to_datetime(y3.index)

fig6 = make_subplots(rows=1, cols=3, subplot_titles=("Confirmed", "Deaths", "Recovered"))

fig6.add_trace(
    go.Scatter(
        x=y1.index, y=y1,
        line=dict(color='blue', width=2),
        name="Confirmed"
    ),
    row=1, col=1
)

fig6.add_trace(
    go.Scatter(
        x=y2.index, y=y2,
        line=dict(color='red', width=2),
        name="Deaths"
    ),
    row=1, col=2
)

fig6.add_trace(
    go.Scatter(
        x=y3.index, y=y3,
        line=dict(color='green', width=2),
        name="Recovered"
    ),
    row=1, col=3
)

fig6 = go.FigureWidget(fig6)
fig6.update_layout(title_text="Daily data on seperate graphs", showlegend=False)

for i in range(3):
    fig6.update_xaxes(title_text="Time", row=1, col=i+1)

fig6.update_yaxes(title_text="Cases", row=1, col=1)
fig6.update_yaxes(rangemode="nonnegative")


container5 = widgets.HBox([country_sel3])
dai_sep = widgets.VBox([container5,
              date_sel3,
              fig6])
dai_sep

## Map Plots

In [11]:
def f4(x):
    if map_sel.value == 'Total Confirmed':
        map_fig['data'][0].z = df_confirmed.iloc[:, -1]
        map_fig['data'][0].colorscale = "Blues"
        
    elif map_sel.value == 'Deadth-to-Confrimed Ratio':
        map_fig['data'][0].z = df_deaths.iloc[:, -1]/df_confirmed.iloc[:, -1]
        map_fig['data'][0].colorscale = "Reds"
        
    elif map_sel.value == 'Recovered-to-Confrimed Ratio':
        map_fig['data'][0].z = df_recovered.iloc[:, -1]/df_confirmed.iloc[:, -1]
        map_fig['data'][0].colorscale = "Greens"

    map_fig.update_layout(title_text = map_sel.value, margin={"r":0,"t":40,"l":0,"b":0}, height=500)

    
    
    
map_sel = widgets.Dropdown(
    options=['Total Confirmed', 'Deadth-to-Confrimed Ratio', 'Recovered-to-Confrimed Ratio'],
    description='Select Plot:',
    value='Total Confirmed',
    disabled=False,
)
map_sel.observe(f4, names='value')

map_fig = go.Figure(go.Choropleth(locationmode = 'country names',
                                        locations=list(df_confirmed['Country']),
                                        z=df_confirmed.iloc[:, -1],
                                        colorscale="Blues",
                                        name='Confirmed'
                                    ))
map_fig = go.FigureWidget(map_fig)
map_fig.update_layout(title_text = 'Total Confirmed', margin={"r":0,"t":40,"l":0,"b":0}, height=500)

maps_w = widgets.VBox([map_sel,
                       map_fig])
maps_w